In [1]:
!pip install mrjob
import re
import pandas as pd


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!curl https://raw.githubusercontent.com/Somu9966/Data-603/main/HomeWork-2/yelp.csv -o yelp.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 7901k  100 7901k    0     0  10.3M      0 --:--:-- --:--:-- --:--:-- 10.3M


In [3]:
read_file = pd.read_csv(r'yelp.csv')
for each_row in range(len(read_file)):
    text = re.sub(r'[\n,]',' ',read_file['text'][each_row])
    read_file.loc[each_row,['text']]=text
    
read_file.to_csv("cleaned_yelp.csv",index_label = False, index=False, header=False)

In [4]:
%%file FirstHW.py
from mrjob.job import MRJob
import re

WORD_RE = re.compile(r"\b\w+\b")

class MRAverageWordCount(MRJob):

    def mapper(self, _, line):
    
            (business_id,date,review_id,stars,text,type,user_id,cool,useful,funny) = line.split(',')
            if (type == "review"):
                words = WORD_RE.findall(text)  #text.split()
                yield (type, len(words))


    def reducer(self, user_id, word_counts):
        total_words = 0
        total_reviews = 0
        for count in word_counts:
            total_words += count
            total_reviews += 1
        yield ("Avg No of words in each review : ", total_words / total_reviews)

if __name__ == '__main__':
    MRAverageWordCount.run()


Overwriting FirstHW.py


In [5]:
import FirstHW

mr_job = FirstHW.MRAverageWordCount(args=['cleaned_yelp.csv'])
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(key, value)

No configs specified for inline runner


Avg No of words in each review :  134.413


In [6]:
%%file FirstHW1.py
from mrjob.job import MRJob
import re

class ReviewCountByYearMonth(MRJob):


    def mapper(self, _, line):
            (business_id,date,review_id,stars,text,type,user_id,cool,useful,funny) = line.split(',')
            match = re.search(r'(\d{4}-\d{2})', date)
            if match:
                year_month = match.group(1)
                yield (year_month, 1)

    def reducer(self, year_month, counts):
        yield (year_month, sum(counts))

if __name__ == '__main__':
    ReviewCountByYearMonth.run()



Overwriting FirstHW1.py


In [7]:
import FirstHW1

mr_job = FirstHW1.ReviewCountByYearMonth(args=['cleaned_yelp.csv'])
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(key, value)

No configs specified for inline runner


2005-04 1
2005-07 2
2005-12 1
2006-01 6
2006-02 9
2006-04 2
2006-05 1
2006-06 5
2006-07 2
2006-08 9
2006-09 4
2006-10 5
2006-11 6
2006-12 6
2007-01 14
2007-02 20
2007-03 42
2007-04 8
2007-05 23
2007-06 12
2007-07 35
2007-08 29
2007-09 26
2007-10 23
2007-11 28
2007-12 25
2008-01 46
2008-02 48
2008-03 47
2008-04 53
2008-05 65
2008-06 76
2008-07 80
2008-08 75
2008-09 59
2008-10 79
2008-11 66
2008-12 71
2009-01 108
2009-02 79
2009-03 126
2009-04 101
2009-05 101
2009-06 67
2009-07 95
2009-08 98
2009-09 113
2009-10 101
2009-11 78
2009-12 104
2010-01 154
2010-02 148
2010-03 168
2010-04 148
2010-05 154
2010-06 118
2010-07 160
2010-08 201
2010-09 150
2010-10 144
2010-11 147
2010-12 160
2011-01 239
2011-02 216
2011-03 263
2011-04 263
2011-05 229
2011-06 230
2011-07 236
2011-08 266
2011-09 193
2011-10 204
2011-11 203
2011-12 249
2012-01 304
2012-02 219
2012-03 259
2012-04 265
2012-05 275
2012-06 272
2012-07 281
2012-08 249
2012-09 239
2012-10 258
2012-11 208
2012-12 196
2013-01 52


In [8]:
%%file FirstHW2.py
from mrjob.job import MRJob

class MRAverageCoolRating(MRJob):

    def mapper(self, _, line):
            (business_id,date,review_id,stars,text,type,user_id,cool,useful,funny) = line.split(',')
            if (int(cool) != 0):
                stars = int(stars)
                yield "Ratings",stars
            

    def reducer(self, key, rating_cool):
        sum_cools = 0
        total_cools = 0
        
        for cool in rating_cool:
            sum_cools+=cool
            total_cools+=1
            
        yield (key,sum_cools/total_cools )

if __name__ == '__main__':
    MRAverageCoolRating.run()


Overwriting FirstHW2.py


In [9]:
import FirstHW2

mr_job = FirstHW2.MRAverageCoolRating(args=['cleaned_yelp.csv'])
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(key, value)

No configs specified for inline runner


Ratings 3.8649595687331537
